## The first go at EDA

In [1]:
import os
import tarfile
import gzip
import io
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/gdrive', force_remount=True)
%cd "/gdrive/MyDrive/Interview Kickstart/MLSwitchup/Capstone/ShopTalk"
%ls

Mounted at /gdrive
/gdrive/MyDrive/Interview Kickstart/MLSwitchup/Capstone/ShopTalk
datasets/  db/  milvus_demo.db  ShopTalkEDA.ipynb


In [4]:
!wget -N -P datasets/ https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-listings.tar
!wget -N -P datasets/ https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar

--2024-11-23 00:30:50--  https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-listings.tar
Resolving amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)... 52.217.230.97, 54.231.172.153, 52.217.230.169, ...
Connecting to amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)|52.217.230.97|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘datasets/abo-listings.tar’ not modified on server. Omitting download.

--2024-11-23 00:30:50--  https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar
Resolving amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)... 52.217.230.97, 54.231.172.153, 52.217.230.169, ...
Connecting to amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)|52.217.230.97|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘datasets/abo-images-small.tar’ not modifi

In [5]:
def read_json_from_tar(tar_file):
    # Open the tar file
    dflist = []
    with tarfile.open(tar_file, 'r') as tar:
        for member in tqdm(tar.getmembers()):
            if member.name.endswith('json.gz'):
                #print(f"Reading {member.name}...")
                # Extract the gz file in memory
                f = tar.extractfile(member)
                if f is not None:
                  with gzip.open(f, 'rt') as f:
                    df = pd.read_json(f, lines=True)
                    dflist.append(df)
    pdf = pd.concat(dflist).set_index('item_id')
    return pdf

pdf = read_json_from_tar("datasets/abo-listings.tar")

100%|██████████| 20/20 [00:32<00:00,  1.65s/it]


In [6]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 147702 entries, B07NQ437BB to B07DFWQV5V
Data columns (total 27 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   brand                147643 non-null  object
 1   bullet_point         131570 non-null  object
 2   color                116180 non-null  object
 3   fabric_type          8193 non-null    object
 4   item_name            147702 non-null  object
 5   item_weight          106194 non-null  object
 6   model_name           81579 non-null   object
 7   model_number         124091 non-null  object
 8   product_type         147702 non-null  object
 9   style                43188 non-null   object
 10  main_image_id        147127 non-null  object
 11  other_image_id       137976 non-null  object
 12  color_code           18243 non-null   object
 13  country              147702 non-null  object
 14  marketplace          147702 non-null  object
 15  domain_name          14770

In [7]:
pdf = pdf.drop(columns=['model_number',
                        'color_code',
                        'node',
                        'item_dimensions',
                        'spin_id',
                        '3dmodel_id',
                        'item_shape'])
pdf = pdf[pdf['country'] == 'US']
pdf = pdf[pdf['domain_name'] == 'amazon.com']
pdf.head()

,brand,bullet_point,color,fabric_type,item_name,item_weight,model_name,product_type,style,main_image_id,other_image_id,country,marketplace,domain_name,material,item_keywords,pattern,model_year,product_description,finish_type
item_id,,,,,,,,,,,,,,,,,,,,
B07LCHFZCW,"[{'language_tag': 'en_US', 'value': 'Amazon Go'}]",NaN,NaN,NaN,"[{'language_tag': 'en_US', 'value': 'Amazon Ki...",NaN,NaN,[{'value': 'GROCERY'}],NaN,61bCuBtuZ4L,"[61njweKAz7L, 61UEtH+yC3L]",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN
B07C2BHFCN,"[{'language_tag': 'en_US', 'value': 'Mama Bear'}]","[{'language_tag': 'en_US', 'value': 'One bottl...",NaN,NaN,"[{'language_tag': 'en_US', 'value': 'Amazon Br...","[{'normalized_value': {'unit': 'pounds', 'valu...",NaN,[{'value': 'HEALTH_PERSONAL_CARE'}],NaN,61oZajNgA-L,"[610areGPMRL, 611qxLJ2P9L, 51anwEUvkZL, 61v8KF...",US,Amazon,amazon.com,NaN,"[{'language_tag': 'en_US', 'value': 'kids vita...",NaN,NaN,NaN,NaN
B07DBJQC1K,"[{'language_tag': 'en_US', 'value': 'Ravenna H...","[{'language_tag': 'zh_CN', 'value': '31.89英寸（约...","[{'language_tag': 'zh_CN', 'standardized_value...",NaN,"[{'language_tag': 'zh_CN', 'value': 'Amazon 品牌...","[{'normalized_value': {'unit': 'pounds', 'valu...",NaN,[{'value': 'CHAIR'}],"[{'language_tag': 'zh_CN', 'value': '过渡'}, {'l...",81CCMOsq9QL,"[915rxibyYsL, 81z-gVKE1fL, A18T8fMcScL, 91wtj8...",US,Amazon,amazon.com,"[{'language_tag': 'zh_CN', 'value': '人造革'}, {'...","[{'language_tag': 'en_US', 'value': 'living-ro...",NaN,NaN,NaN,NaN
B08B8XXZNM,"[{'language_tag': 'en_US', 'value': 'Rivet'}]","[{'language_tag': 'en_US', 'value': 'The soft,...","[{'language_tag': 'en_US', 'standardized_value...",NaN,"[{'language_tag': 'en_US', 'value': 'Amazon Br...","[{'normalized_value': {'unit': 'pounds', 'valu...",NaN,[{'value': 'CHAIR'}],NaN,91A0WF7ZG5L,"[91xrOSOlUCL, A1r8gSRwQbL, 71othXV15wL, 91sQzq...",US,Amazon,amazon.com,NaN,"[{'language_tag': 'en_US', 'value': 'makeup'},...",NaN,NaN,NaN,NaN
B07M7NSYD8,"[{'language_tag': 'en_US', 'value': 'Ravenna H...","[{'language_tag': 'en_US', 'value': '30""W x 22...","[{'language_tag': 'en_US', 'standardized_value...",NaN,"[{'language_tag': 'en_US', 'value': 'Ravenna H...","[{'normalized_value': {'unit': 'pounds', 'valu...",NaN,[{'value': 'HOME_FURNITURE_AND_DECOR'}],NaN,71Veq8BJy+L,"[71PcCUIAmcL, 91qUaSyXElL, 71nZbSENTuL, 21ISGz...",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
import numpy as np

def extract_value(value):
  if isinstance(value, list) and all(isinstance(item, dict) for item in value):
    for item in value:
      if 'language_tag' in item and item['language_tag'].startswith('en_'):
        return item['value']
      if 'value' in item:
        return item['value']
  return value

for col in tqdm(pdf.columns):
  for i in range(len(pdf[col])):
    pdf[col][i] = extract_value(pdf[col][i])
pdf.head()

100%|██████████| 20/20 [03:27<00:00, 10.35s/it]


,brand,bullet_point,color,fabric_type,item_name,item_weight,model_name,product_type,style,main_image_id,other_image_id,country,marketplace,domain_name,material,item_keywords,pattern,model_year,product_description,finish_type
item_id,,,,,,,,,,,,,,,,,,,,
B07LCHFZCW,Amazon Go,NaN,NaN,NaN,"Amazon Kitchen, Cajun Style Potato Salad, 7.2 oz",NaN,NaN,GROCERY,NaN,61bCuBtuZ4L,"[61njweKAz7L, 61UEtH+yC3L]",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN
B07C2BHFCN,Mama Bear,"One bottle of 60 multivitamin gummies, a 1 mon...",NaN,NaN,Amazon Brand - Mama Bear Organic Kids Multivit...,0.37,NaN,HEALTH_PERSONAL_CARE,NaN,61oZajNgA-L,"[610areGPMRL, 611qxLJ2P9L, 51anwEUvkZL, 61v8KF...",US,Amazon,amazon.com,NaN,kids vitamins,NaN,NaN,NaN,NaN
B07DBJQC1K,Ravenna Home,31.89英寸（约80厘米）长 x 31.89英寸（约80厘米）宽 x 33.86英寸（约8...,米色,NaN,Amazon 品牌 – Ravenna Home Christian 中世纪现代风格椅子，3...,46,NaN,CHAIR,过渡,81CCMOsq9QL,"[915rxibyYsL, 81z-gVKE1fL, A18T8fMcScL, 91wtj8...",US,Amazon,amazon.com,人造革,living-room-chairs,NaN,NaN,NaN,NaN
B08B8XXZNM,Rivet,"The soft, curved lines of this adjustable offi...",Grey,NaN,Amazon Brand – Rivet Modern Upholstered Swivel...,39.9,NaN,CHAIR,NaN,91A0WF7ZG5L,"[91xrOSOlUCL, A1r8gSRwQbL, 71othXV15wL, 91sQzq...",US,Amazon,amazon.com,NaN,makeup,NaN,NaN,NaN,NaN
B07M7NSYD8,Ravenna Home,"30""W x 22""D x 36.5""H",Black,NaN,"Ravenna Home 2-Drawer Kitchen Cart, 36.5""H, Bl...",64.82,NaN,HOME_FURNITURE_AND_DECOR,NaN,71Veq8BJy+L,"[71PcCUIAmcL, 91qUaSyXElL, 71nZbSENTuL, 21ISGz...",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
pdf = pdf.groupby('product_type').sample(n=10, random_state=1, replace=True)
pdf.head()

,brand,bullet_point,color,fabric_type,item_name,item_weight,model_name,product_type,style,main_image_id,other_image_id,country,marketplace,domain_name,material,item_keywords,pattern,model_year,product_description,finish_type
item_id,,,,,,,,,,,,,,,,,,,,
B07JD3PQJR,Eono,NaN,Makeup Brush Cleaner,NaN,Eono Essentials Makeup Brush Cleaner Cleaning ...,NaN,NaN,ABIS_BEAUTY,NaN,61LVc++Fn9L,"[51YlAUrN2OL, 61sQsElOFIL, 61fHfTpueHL, 619mZx...",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN
B07JD3PQJR,Eono,NaN,Makeup Brush Cleaner,NaN,Eono Essentials Makeup Brush Cleaner Cleaning ...,NaN,NaN,ABIS_BEAUTY,NaN,61LVc++Fn9L,"[51YlAUrN2OL, 61sQsElOFIL, 61fHfTpueHL, 619mZx...",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN
B0876P49KD,Solimo,Amazon Brand: Solimo Body Wash - Sensitive Pac...,NaN,NaN,Amazon Brand: Solimo Body Wash - Sensitive Pac...,3.496,NaN,ABIS_BEAUTY,NaN,51lqn30MtmL,"[61Au5ZF9+LL, 61MjNooTq6L, 61rzYOCn-WL, 51qauH...",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN
B0876P49KD,Solimo,Amazon Brand: Solimo Body Wash - Sensitive Pac...,NaN,NaN,Amazon Brand: Solimo Body Wash - Sensitive Pac...,3.496,NaN,ABIS_BEAUTY,NaN,51lqn30MtmL,"[61Au5ZF9+LL, 61MjNooTq6L, 61rzYOCn-WL, 51qauH...",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN
B07JD3PQJR,Eono,NaN,Makeup Brush Cleaner,NaN,Eono Essentials Makeup Brush Cleaner Cleaning ...,NaN,NaN,ABIS_BEAUTY,NaN,61LVc++Fn9L,"[51YlAUrN2OL, 61sQsElOFIL, 61fHfTpueHL, 619mZx...",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
pdf.to_pickle("datasets/abo-listings.pkl")
pdf = pd.read_pickle("datasets/abo-listings.pkl")
pdf.head()

,brand,bullet_point,color,fabric_type,item_name,item_weight,model_name,product_type,style,main_image_id,other_image_id,country,marketplace,domain_name,material,item_keywords,pattern,model_year,product_description,finish_type
item_id,,,,,,,,,,,,,,,,,,,,
B07JD3PQJR,Eono,NaN,Makeup Brush Cleaner,NaN,Eono Essentials Makeup Brush Cleaner Cleaning ...,NaN,NaN,ABIS_BEAUTY,NaN,61LVc++Fn9L,"[51YlAUrN2OL, 61sQsElOFIL, 61fHfTpueHL, 619mZx...",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN
B07JD3PQJR,Eono,NaN,Makeup Brush Cleaner,NaN,Eono Essentials Makeup Brush Cleaner Cleaning ...,NaN,NaN,ABIS_BEAUTY,NaN,61LVc++Fn9L,"[51YlAUrN2OL, 61sQsElOFIL, 61fHfTpueHL, 619mZx...",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN
B0876P49KD,Solimo,Amazon Brand: Solimo Body Wash - Sensitive Pac...,NaN,NaN,Amazon Brand: Solimo Body Wash - Sensitive Pac...,3.496,NaN,ABIS_BEAUTY,NaN,51lqn30MtmL,"[61Au5ZF9+LL, 61MjNooTq6L, 61rzYOCn-WL, 51qauH...",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN
B0876P49KD,Solimo,Amazon Brand: Solimo Body Wash - Sensitive Pac...,NaN,NaN,Amazon Brand: Solimo Body Wash - Sensitive Pac...,3.496,NaN,ABIS_BEAUTY,NaN,51lqn30MtmL,"[61Au5ZF9+LL, 61MjNooTq6L, 61rzYOCn-WL, 51qauH...",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN
B07JD3PQJR,Eono,NaN,Makeup Brush Cleaner,NaN,Eono Essentials Makeup Brush Cleaner Cleaning ...,NaN,NaN,ABIS_BEAUTY,NaN,61LVc++Fn9L,"[51YlAUrN2OL, 61sQsElOFIL, 61fHfTpueHL, 619mZx...",US,Amazon,amazon.com,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
def extract_image_metadata_from_tar(tar_file):
    # Open the tar file
    with tarfile.open(tar_file, 'r') as tar:
        for member in tqdm(tar.getmembers()):
            if member.name.endswith('images/metadata/images.csv.gz'):
                # Extract the gz file in memory
                f = tar.extractfile(member)
                if f is not None:
                  with gzip.open(f, 'rb') as f_in:
                    output_file_name = tar_file[:-len('.tar')] + '/' +member.name[:-len('.gz')]
                    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)
                    with open(output_file_name, 'wb') as f_out:
                      f_out.write(f_in.read())
                      return f_out.name

image_meta_path = extract_image_metadata_from_tar("datasets/abo-images-small.tar")
print(image_meta_path)

100%|█████████▉| 398473/398474 [00:00<00:00, 985543.33it/s] 

datasets/abo-images-small/images/metadata/images.csv


In [29]:
image_meta_df = pd.read_csv(image_meta_path).set_index('image_id')
image_meta_df.head()

,height,width,path
image_id,,,
010-mllS7JL,106,106,14/14fe8812.jpg
01dkn0Gyx0L,122,122,da/daab0cad.jpg
01sUPg0387L,111,111,d2/d2daaae9.jpg
1168jc-5r1L,186,186,3a/3a4e88e6.jpg
11RUV5Fs65L,30,500,d9/d91ab9cf.jpg


In [30]:
all_image_paths = set()
for main_image_id in pdf['main_image_id']:
  if main_image_id in image_meta_df.index:
    all_image_paths.add(image_meta_df.loc[main_image_id]['path'])
for other_image_id_list in pdf['other_image_id']:
  if isinstance(other_image_id_list, list):
    for other_image_id in other_image_id_list:
      if other_image_id in image_meta_df.index:
        all_image_paths.add(image_meta_df.loc[other_image_id]['path'])
print(len(all_image_paths))

9279


In [32]:
def extract_images_from_tar(tar_file):
    # Open the tar file
    with tarfile.open(tar_file, 'r') as tar:
        for member in tqdm(tar.getmembers()):
            image_path = member.name[len('images/small/'):]
            if image_path in all_image_paths:
                target_path = tar_file[:-len('.tar')] + '/' + image_path
                tar.extract(member, path=target_path)

extract_images_from_tar("datasets/abo-images-small.tar")

100%|██████████| 398474/398474 [02:41<00:00, 2473.76it/s]
